>Ouvrir le notebook dans Colab en modifiant le début de son adresse dans le navigateur :<br>
il faut remplacer **github.com** par **githubtocolab.com**.<br>
Une fois vos réponses apportées, le notebook devra être sauvegardé dans GitHub, dans le repository du TP :<br>
*Fichier > Sauvegarder une copie dans Github*<br>

---

# Un peu de géographie

In [ ]:
# imports
import pandas as pd # bibliothèques dédiée au traitement de jeux de données
import matplotlib.pyplot as plt # bibliothèque graphique
import seaborn as sns # bibliothèque graphique reposant sur matplotlib et dédiée plus particulièrement à la représentation de jeux de données

# données
url = "https://raw.githubusercontent.com/Info-TSI-Vieljeux/s1-tp3/main/data_monde_classe.csv"
data_monde = pd.read_csv(url,index_col=0)
data_monde.head(3)

Certains des modules utiles n'étant pas présent dans Colabotary, il faut les installer :

In [ ]:
# Installations utiles à Geopandas
!apt install gdal-bin python-gdal python3-gdal 
!apt install python3-rtree 
# Geopandas (permet de traiter les données géographiques dans une dataframe pandas)
!pip install -q git+git://github.com/geopandas/geopandas.git
# Autres packages nécessaires à Geopandas
!pip install -q contextily
!pip install -q mapclassify

In [ ]:
import contextily as cx
import mapclassify
import geopandas as gpd

On récupère un jeu de données géométrico-géographique comprenant la forme de chaque pays (ensemble de points dessinant un polygone) pour pouvoir construire une carte.

In [ ]:
url = "http://opendata.arcgis.com/datasets/a21fdb46d23e4ef896f31475217cbb08_1.geojson"
localisation = gpd.read_file(url)
localisation.head(3)

La colonne "geometry" contient le tracé des pays.<br>
On joint les formes géométriques à notre jeu de données originel.

In [ ]:
data_geoloc = localisation.set_index('CNTRY_NAME').join(data_monde)
data_geoloc.head(3)

On retire l'Antartique et on choisit une formule de projection pour représenter la carte (le WGS 84 des GPS ici).

In [ ]:
monde = data_geoloc.to_crs("+proj=robin")
monde = monde[monde.index != "Antarctica"]
monde = monde[monde.index != "Greenland"]

In [ ]:
ax = monde.plot(figsize = (30,10), alpha=0.8, column = "Score de bonheur", cmap = "viridis", legend=True, legend_kwds={"label":"Score de bonheur"},missing_kwds={'color': 'lightgrey'})
ax.axis('off')

On est maintenant paré pour représenter les 3 clusters obtenus par l'algo des k-moyennes du TP3b.

In [ ]:
monde["Cluster"]=monde["Cluster"].astype('Int64')
ax = monde.plot(figsize = (30,10), alpha=0.8, column = "Cluster", cmap = "viridis", legend=True, missing_kwds={'color': 'lightgrey'},categorical=True)
ax.axis('off')

Fabriquons une carte régionale en ne sélectionnant que certaines régions du monde.

In [ ]:
region = data_geoloc[data_geoloc["Région du monde"].isin(["Western Europe","Central and Eastern Europe","Middle East and North Africa"])]
region = region.to_crs(epsg=3857) # utilise des mètres à la place des degrés
palette = sns.diverging_palette(150, 275, n=20, as_cmap=True) # dégradé pour remplacer 'viridis'

In [ ]:
ax = region.plot(figsize = (20,20),alpha=0.8,column = "Score de bonheur", cmap=palette, edgecolor='w', legend=True, legend_kwds={"label":"Score de bonheur",'shrink': 0.6})
cx.add_basemap(ax, url=cx.providers.Stamen.Watercolor) # parcequ'on peut...
ax.axis('off')

Modifier les cellules qui précèdent pour que le graphique ci-dessus affiche la carte du PIB par habitant des pays d'Asie du sud.

In [ ]:
# Cellule de vérification (ne pas modifier)

De quelle couleur est le Vietnam sur cette carte ?

In [ ]:
# Effacer les deux réponses fausses
couleur1 = "vert"
couleur1 = "rose"
couleur1 = "violet"

In [ ]:
# Cellule de vérification (ne pas modifier)

On peut aussi jouer avec des cartes interactives.

In [ ]:
import folium

In [ ]:
monde = data_geoloc.to_crs(epsg=4326)
mondo = monde.dissolve(by='CNTRY_NAME').reset_index()
tuile = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}'

In [ ]:
carte = folium.Map(location=[46, -1],
               tiles=tuile,
               attr='Tiles &copy; Esri &mdash; Sources: GEBCO, NOAA, CHS, OSU, UNH, CSUMB, National Geographic, DeLorme, NAVTEQ, and Esri',
               min_zoom=2, 
               max_zoom=6, 
               zoom_start=4)

folium.Choropleth(mondo,                                
                  data=mondo,                           
                  key_on='feature.properties.CNTRY_NAME', 
                  columns=['CNTRY_NAME', 'Générosité'],  
                  fill_color='PiYG_r',
                  bins=11,
                  nan_fill_color='lightgrey',
                  line_weight=0.1,                       
                  line_opacity=0.5,                     
                  legend_name='Score de bonheur').add_to(carte) 

carte

Modifiez la carte précédente pour qu'elle fasse apparaître le score de générosité.<br> 
De quelle couleur apparaît le Kénya sur cette nouvelle carte ?

In [ ]:
# Effacer les deux réponses fausses
couleur2 = "vert"
couleur2 = "rose"
couleur2 = "violet"

In [ ]:
# Cellule de vérification (ne pas modifier)